In [2]:
!pip3 install tweepy
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 19.1 MB/s eta 0:00:00


In [3]:
import tweepy
import time
from kafka import KafkaProducer

# Update the keys with correct values
bearer_token= r"AAAAAAAAAAAAAAAAAAAAAKgwhQEAAAAARkzo4dqWxtEWPzz9rPvTrjoNXpE%3DdaTgGJdbavPOvjOophZ6F7Q2bz0SA4UUxzs3LwWdXxdLX68AHz"
api_key = "xY9EdDNhhiUxRcYR2VkoDYy3z"
api_secret = "cvLBHSMalhaO0NirqPzvvR2IupHBkA4XHSTqeB8QF22PW9wL6g"
access_token = '1572264324764147720-Jjz7IanGYwU39CJFItgXQUVIjzMFCp'
access_token_secret = '2sZVcrkz1Hvtw9HDxlyd4SML1Pwy3S5oFqzSj37kgTKJl'

# from "targets" var in "Project Checkpoint 1" notebook
search_terms = ['slowite', 'bruteforce', 'flood', 'malformed', 'dos', 'legitimate']

producer = KafkaProducer(bootstrap_servers='localhost:9092')

topic_name = "project-twitter-streaming"

# set timer for 5 mins from now
five_min_from_now = time.time() + 300

print(five_min_from_now)

class MyStream(tweepy.StreamingClient):
    def on_connect(self):
        print("connected")
    
    def on_tweet(self,tweet):
        # disconnect stream after 5 mins
        if time.time() > five_min_from_now:
            print('disconnecting')
            self.disconnect()
            
        if tweet.referenced_tweets == None:
            print(tweet.text)
            producer.send(topic_name,tweet.text.encode('utf-8'))
            time.sleep(0.1)
            
            
stream = MyStream(bearer_token=bearer_token)


# first delete all existing Tweepy streaming rules if rules exist
ids = []
rules = list(stream.get_rules())[0]

if rules != None:
    for rule in list(stream.get_rules())[0]:
        ids.append(rule[2])

    stream.delete_rules(ids)

# add rules
for term in search_terms:
    print(term)
    stream.add_rules(tweepy.StreamRule(term))
    
stream.filter(tweet_fields=["referenced_tweets"])

NoBrokersAvailable: NoBrokersAvailable